In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import Imputer
import numpy as np
from itertools import groupby
from math import log
%matplotlib inline

In [3]:
properties = pd.read_csv('data/props/datos_procesados.csv')
properties.dropna(subset=['id','price','property_type', 'state_name', 'place_name', 'description'], inplace=True)
properties = properties.loc[properties.created_on.str.contains('2016') | properties.created_on.str.contains('2017'), :]

/Users/mreverter/PythonProjects/datos/tp-last/env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,9,25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
properties = properties.loc[properties.price < properties.price.quantile(0.85), :]

In [5]:
def process(word):
    word = word.replace(",", "")
    word = word.replace(".", "")
    word = word.replace("?", "")
    word = word.replace("!", "")
    word = word.replace('"', '')
    word = word.replace('*', '')
    word = word.replace('`', '')
    word = word.lower()
    word = word.replace('�', '')
    word = word.replace('#', '')
    word = word.replace('', '')
    word = word.strip()
    return word

## Generación de precios promedio por palabra

In [6]:
properties['words'] = properties.apply(lambda x: x.description.split(' '), axis=1)

In [7]:
wordsAndPrices = properties.loc[:,['words', 'price']]

In [8]:
wordsPrices = map(lambda x: [(word, x['price']) for word in x.words], wordsAndPrices.to_records())

In [9]:
wordsPrices = [item for sublist in wordsPrices for item in sublist]

In [10]:
wordsPrices = map(lambda x: (process(x[0]), x[1]), wordsPrices)

In [11]:
wordsPrices = filter(lambda x: len(x[0]) >= 4, wordsPrices)

In [12]:
df = pd.DataFrame(wordsPrices)

In [13]:
df.columns = ['word', 'price']

In [14]:
grouped = pd.DataFrame(df.groupby('word').agg([np.mean, np.size]).to_records())

In [15]:
grouped.columns = ['word', 'precio', 'apariciones']

In [16]:
grouped = grouped.loc[(grouped['apariciones'] > 1000) & (grouped['apariciones'] < 41000), :]

In [17]:
grouped.sort_values(by='precio', ascending=False, inplace=True)

In [21]:
grouped

,word,precio,apariciones
140667,family,436143.358675,2144.0
93992,chimenea,416075.308008,1948.0
221962,riego,402049.123401,2893.0
142538,financiado,394613.722791,1075.0
167137,laguna,387773.988858,2154.0
163741,isla,386931.673866,1852.0
115765,dependencias,386089.644756,2069.0
100034,comparten,381833.996562,2036.0
146703,galerÃ­a,380202.911662,2298.0
217486,recepciÃ³n,375723.107094,1466.0


In [19]:
grouped.to_csv('words_price.csv')

## Corrección de predicciones

In [22]:
wordsPriceDf = pd.read_csv('words_price.csv')

In [20]:
def correctByDescription(pricePredicted, description, referencePricesDf, factor):
    price = 0
    descWords = description.split(' ')
    descWords = [process(a) for a in descWords]
    descWords = [a for a in descWords if len(a) >= 4]
    descWords = [a for a in descWords if len(referencePricesDf[referencePricesDf['word'] == a]) != 0]
    for word in descWords:
        price = price + referencePricesDf[referencePricesDf['word'] == word].iloc[0].precio        
    if len(descWords) == 0:
        return pricePredicted
    else:
        return pricePredicted + (((price / len(descWords)) - pricePredicted)  * factor)

In [23]:
def correctPredictions(predicciones, descriptions, factor):
    predictionsCorrected = []
    i=0
    size = len(predicciones)
    for idx, prediction in enumerate(predicciones):
        i=i+1
        predictionsCorrected.append(correctByDescription(prediction, descriptions.iloc[idx].description, wordsPriceDf, factor))
    return predictionsCorrected

Factor pertenece a [0,1]